<a href="https://colab.research.google.com/github/RodolfoFerro/PyConCo20/blob/full-code/notebooks/Deep%20Learning%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring the data

This first section contains the utility functions to load the dataset to be used for our model training.

The dataset used for this project is the one published in the "[Challenges in Representation Learning: Facial Expression Recognition Challenge](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data)" by Kaggle.

> **"Challenges in Representation Learning: A report on three machine learning contests."** I. Goodfellow, D. Erhan, P. L. Carrier, A. Courville, M. Mirza, B. Hamner, W. Cukierski, Y. Tang, D. H. Lee, Y. Zhou, C. Ramaiah, F. Feng, R. Li, X. Wang, D. Athanasakis, J. Shawe-Taylor, M. Milakov, J. Park, R. Ionescu, M. Popescu, C. Grozea, J. Bergstra, J. Xie, L. Romaszko, B. Xu, Z. Chuang, and Y. Bengio. arXiv 2013.

## Data description (as detailed by Kaggle)

The data consists of 48x48 pixel grayscale images of faces. The faces have been automatically registered so that the face is more or less centered and occupies about the same amount of space in each image. The task is to categorize each face based on the emotion shown in the facial expression in to one of seven categories (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral).

The csv file contains two main columns, "emotion" and "pixels". The "emotion" column contains a numeric code ranging from 0 to 6, inclusive, for the emotion that is present in the image. The "pixels" column contains a string surrounded in quotes for each image. The contents of this string a space-separated pixel values in row major order. test.csv contains only the "pixels" column and your task is to predict the emotion column.

This dataset was prepared by Pierre-Luc Carrier and Aaron Courville, as part of an ongoing research project. They have graciously provided the workshop organizers with a preliminary version of their dataset to use for this contest.

First we need to be sure that we are using the laest version of TensorFlow:

In [ ]:
!pip install -q tensorflow==2.0.0 tensorboard==2.0.0

In [ ]:
import tensorflow as tf
tf.__version__

## The dataset loader

This function will allow us to load directly all the dataset using pandas. This will also split the dataset into training and testing subsets.

In [ ]:
from tensorflow import keras
import pandas as pd
import numpy as np


def load_dataset(net=True):
    """Utility function to load the FER2013 dataset.
    
    It returns the formated tuples (X_train, y_train) , (X_test, y_test).

    Parameters
    ==========
    net : boolean
        This parameter is used to reshape the data from images in 
        (cols, rows, channels) format. In case that it is False, a standard
        format (cols, rows) is used.
    """

    # Load and filter in Training/not Training data:
    df = pd.read_csv('../data/fer2013.csv')
    training = df.loc[df['Usage'] == 'Training']
    testing = df.loc[df['Usage'] != 'Training']

    # X_train values:
    X_train = training[['pixels']].values
    X_train = [np.fromstring(e[0], dtype=int, sep=' ') for e in X_train]
    if net:
        X_train = [e.reshape((48, 48, 1)).astype('float32') for e in X_train]
    else:
        X_train = [e.reshape((48, 48)) for e in X_train]
    X_train = np.array(X_train)

    # X_test values:
    X_test = testing[['pixels']].values
    X_test = [np.fromstring(e[0], dtype=int, sep=' ') for e in X_test]
    if net:
        X_test = [e.reshape((48, 48, 1)).astype('float32') for e in X_test]
    else:
        X_test = [e.reshape((48, 48)) for e in X_test]
    X_test = np.array(X_test)

    # y_train values:
    y_train = training[['emotion']].values
    y_train = keras.utils.to_categorical(y_train)

    # y_test values
    y_test = testing[['emotion']].values
    y_test = keras.utils.to_categorical(y_test)

    return (X_train, y_train) , (X_test, y_test)

## Data loading

If you have not downloaded the dataset yet, run the following cell. If you have already done this, you can skip this step.

In [ ]:
!mkdir ../data
!wget -O ../data/fer2013.csv https://www.dropbox.com/s/zi48lkarsg4kbry/fer2013.csv\?dl\=1

We can now use our main function to load the complete dataset.

In [ ]:
(X_train, y_train) , (X_test, y_test) = load_dataset()

We can access the data and plot some samples to check out waht's inside:

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

plt.figure(figsize=(12, 12))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(X_train[i].reshape((48, 48)), cmap="gray")
    plt.axis('off')
    plt.tight_layout()

And, to understand how each image is loaded, we can print any element as a matrix:

In [ ]:
X_train[i].reshape((48, 48))

# Creating our Deep Learning model

Now the interesting part comes to us. Let's build a custom DL model.

## Model architecture

After some research in the state of the art for Facial Expression Recognition, I found that a simple convolutional architecture based on LeNet-5 would achieve nice results. 

Anyway, more recent proposals have achieved more accurate results, and even if Tensorflow already includes prebuilt models (such as MobileNet, which is one of the best model architectures for portable devices), I came up with my own implementation based on a neetwrk architecture which is supposed to be a deep-lightweight accurate model for the FER problem: "[Extended deep neural network for facial emotion recognition (EDNN)](https://www.sciencedirect.com/science/article/abs/pii/S016786551930008X)" by Deepak Kumar Jaina, Pourya Shamsolmoalib, and Paramjit Sehdev (Elsevier – Pattern Recognition Letters 2019).

In their paper, they assure through some tests that their EDNN gives better results in classification tasks for Facial Expression Recognition, and by the architecture metrics this network turns out to be a more lightweight model compared with others.

In any case, I proceeded to use Tensorflow 2.0 to build my own EDNN implementation with the Keras module.

The implementation will come from two functions:
- One to build the Residual Block
- The second one to build the rest of the model

The residual block architecture is as follows:

<center>
    <img src="https://raw.githubusercontent.com/RodolfoFerro/PyConCo20/full-code/media/residual_block.png" width="25%">
</center>

We can now proceed to build our EDNN model:

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import SGD


def ResidualBlock(prev_layer):
    """Residual block from the EDNN model for FER by Deepak Kumar Jaina,
    Pourya Shamsolmoalib & Paramjit Sehdev, as it appears in "Extended 
    deep neural network for facial emotion recognition", 2019.
    """
    
    # conv_1 <- Conv2D(64, (1, 1)) on prev_layer
    # conv_2 <- Conv2D(64, (3, 3), padding="same") on conv_1
    shortc = concatenate([conv_1, conv_2], axis=-1)
    # conv_3 <- Conv2D(128, (3, 3), padding="same") on shortc
    # conv_4 <- Conv2D(256, (1, 1)) on conv_3
    output = concatenate([conv_4, prev_layer], axis=-1)
    
    return output


def EDNN(n_classes=7):
    """
    EDNN model for FER by Deepak Kumar Jaina, Pourya Shamsolmoalib &
    Paramjit Sehdev, as it appears in "Extended deep neural network for 
    facial emotion recognition", 2019.
    """

    x = Input(shape=(48, 48, 1))
    y = Conv2D(32, (5, 5), input_shape=(48, 48, 1), strides=(2, 2), 
               data_format='channels_last')(x)
    # y <- MaxPooling2D(pool_size=(2, 2)) on y
    # y <- Conv2D(64, (3, 3), strides=(1, 1)) on y
    # y <-  ResidualBlock on y
    # y <- Conv2D(128, (3, 3), strides=(1, 1), padding="same") on y
    # y <- MaxPooling2D(pool_size=(2, 2)) on y
    # y <- Conv2D(128, (3, 3), strides=(1, 1)) on y
    # y <- ResidualBlock on y
    # y <- Conv2D(256, (3, 3), strides=(1, 1), padding="same") on y
    # y <- MaxPooling2D(pool_size=(2, 2)) on y
    # y <- Conv2D(512, (3, 3), strides=(1, 1), padding="same") on y
    # y <- Flatten() on y
    # y <- Dense(1024, activation='relu') on y
    # y <- Dropout(0.2) on y
    # y <- Dense(512, activation='relu') on y
    # y <- Dropout(0.2) on y
    # y <- Dense(n_classes, activation='softmax') on y
    
    # Create model
    model = Model(x, y)

    # Compile model
    opt = SGD(lr=LRATE, momentum=0.9, decay=LRATE/EPOCHS)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt, metrics=['accuracy'])

    return model

We now create an instance of our model and verify the details of the architecture:

In [ ]:
# Set hyperparameters
EPOCHS = 1
BATCH = 64
LRATE = 1e-4

# Instance the model
ednn = EDNN()

In [ ]:
ednn.summary()

## Model training

So far we have created out model and we already have loaded the datset. But beofr, let's create our media folder:

In [ ]:
!mkdir ../media

We can now proceed to train the model.

In [ ]:
history = ednn.fit(X_train, y_train,
                   validation_data=(X_test, y_test),
                   epochs=EPOCHS, batch_size=BATCH)

After the training, we can plot the history of this process.

The following functions will allow us to do so.

In [ ]:
def plot_loss(history):
    plt.style.use("ggplot")
    plt.figure(figsize=(8, 4))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title("Model's training loss")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()


def plot_accuracy(history):
    plt.style.use("ggplot")
    plt.figure(figsize=(8, 4))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title("Model's training accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Accuracy")
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [ ]:
# Plot loss
plot_loss(history)
# plt.savefig('../media/loss.png', dpi=300)

# Plot accuracy
plot_accuracy(history)
# plt.savefig('../media/accuracy.png', dpi=300)

A way to explore accuracy through classes is using a confusion matrix:

In [ ]:
# Create emotions map
emotion_labels = [
    'Angry',
    'Disgust',
    'Fear',
    'Happy',
    'Sad',
    'Surprise',
    'Neutral'
]

# Predict using trained model
y_pred = ednn.predict(X_test)
y_pred = np.asarray([np.argmax(e) for e in y_pred])
y_true = np.asarray([np.argmax(e) for e in y_test])

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns


# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)
cm_normalised = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot confusion matrix
sns.set(font_scale=1.5) 
fig, ax = plt.subplots(figsize=(10,10))
ax = sns.heatmap(cm_normalised, annot=True, linewidths=0, square=False, 
                 cmap='gray', yticklabels=emotion_labels,
                 xticklabels=emotion_labels, vmin=0,
                 vmax=np.max(cm_normalised), fmt=".2f",
                 annot_kws={"size": 20})
ax.set(xlabel='Predicted label', ylabel='True label')

## Saving the trained model

To save the trained model we will basically do two things:

- Serialize the model into a JSON file, which will save the architecture of our model.
- Serialize the weights into a HDF5 file, which will save all parameters of our model.

In [ ]:
# Serialize model to JSON
json_ednn = ednn.to_json()
with open('../data/model.json', 'w') as json_file:
    json_file.write(json_ednn)

# Serialize weights to HDF5 (h5py needed)
ednn.save_weights('../data/model.h5')
print('Model saved to disk.')

> ### ❗️ If youre using Google Colab only!

## Downloading the saved model

We just need to import the Google Colab module and download the specified files.

In [ ]:
from google.colab import files

model_files = ['../data/model.json', '../data/model.h5']
for file in model_files:
    files.download(file)